### First experiment: Iterative Fibonacci.
Codes obtained from [http://rosettacode.org/wiki/Fibonacci_sequence] for Python and Julia. 
We will use the codes:
* C++ (Iterative version)
* Python (Iterative version)
* Julia (Iterative version

Iterations are based on:
$(x,y) \gets (y,x+y)$

#### For convenience, C++ code:
```
// iterative version
int Fibonacci(int n)
{
	int x = 0;
	int y = 1;
	int t;
	for(int i=1; i<n; ++i)
	{
	    t = x;
	    x = y;
	    y = t+y;
	}
	return y;
}
```

#### For convenience, Python code:
```
def fibIter(n):
    if n < 2:
        return n
    fibPrev = 1
    fib = 1
    for num in xrange(2, n):
        fibPrev, fib = fib, fib + fibPrev
    return fib
```

#### For convenience, Julia code:
```
function fib(n)
  x,y = (0,1)
  for i = 1:n x,y = (y, x+y) end
  x
end
```

In [53]:
# run julia code here:
function fib(n)
  x,y = (0,1)
  for i = 1:n x,y = (y, x+y) end
  x
end

fib (generic function with 1 method)

In [66]:
const f = 0
@time for i = 1:10
    f = fib(20)
end
@show f

  0.000020 seconds (10 allocations: 160 bytes)
f = 6765


6765

Link to [Python notebook](Python_codes.ipynb)

Link to [C++ code](Fib.cpp)
Run:
```
$g++ Fib.cpp
$./a.out 20
```

# Next, we want to write wrappers for both of the Python and C++ codes

### starting with C++ code:
    * copy the function Fibonacci to a new file, add `extern "C"` to your code. The new code should be:
```
#include <stdint.h>

extern "C"
{
int64_t Fibonacci(int64_t n)
{
	int x = 0;
	int y = 1;
	int t;
	for(int i=1; i<n; ++i)
	{
	    t = x;
	    x = y;
	    y = t+y;
	}
	return y;
}
};
```
    * create a makefile to generate the shared libraries. Makefile code would look like:
```
UNAME := $(shell uname)
ifeq ($(UNAME),Linux)
	SHAREDLIB_EXT := so
	SHAREDLIB_FLAG := -shared
endif
ifeq ($(UNAME),Darwin)
	SHAREDLIB_EXT := dylib
	SHAREDLIB_FLAG := -dynamiclib
endif

CURDIR := $(shell pwd)
FIBOLIBNAME := $(CURDIR)/Fibo_externC.$(SHAREDLIB_EXT)

CXXFLAGS:= -std=c++0x -U__STRICT_ANSI__ -O3 -fPIC -c
OBJS=Fibo_externC.o
all: fibonacci

.PHONY: all clean

fibonacci: Fibo_externC.cpp
	g++ $(CXXFLAGS) Fibo_externC.cpp
	g++ $(SHAREDLIB_FLAG) -o $(FIBOLIBNAME) Fibo_externC.o
        # used to use this to fix lib paths:
        # export LD_LIBRARY_PATH=$(CURDIR):$$LD_LIBRARY_PATH
	
clean:
	$(RM) -rf $(OBJS)
	$(RM) -rf $(FIBOLIBNAME)
```

In [7]:
if OS_NAME == :Linux
    const libpath = string(pwd(), "/Fibo_externC.so")
else
    const libpath = string(pwd(), "/Fibo_externC.dylib")
end

"/Users/hudanassar/Documents/A-Research/Presentations/PyDataChi2016/3-wrappers_and_speed/Fibo_externC.dylib"

In [8]:
function cpp_wrapped_fibo(n::Int64)
    r = ccall( (:Fibonacci_I, libpath), # function + library
                    Int64, # return type
                    (Int64,),
                    n)
    return r
end

cpp_wrapped_fibo (generic function with 1 method)

In [14]:
const h = 0
@time for i = 1:10
    h = cpp_wrapped_fibo(20)
end
@show h

  0.000015 seconds (10 allocations: 160 bytes)
f = 6765


6765

### Wrapper for Python code now:
* Create file fibo.py
```
def fibIter(n):
    if n < 2:
        return n
    fibPrev = 1
    fib = 1
    for num in xrange(2, n):
        fibPrev, fib = fib, fib + fibPrev
    return fib
```

In [1]:
using PyCall #if calling this for the first time, add the package first by typing Pkg.add("PyCall")
unshift!(PyVector(pyimport("sys")["path"]), "") #to tell pyimport search in the local directory

PyObject ['', '/Users/hudanassar/anaconda/lib/python27.zip', '/Users/hudanassar/anaconda/lib/python2.7', '/Users/hudanassar/anaconda/lib/python2.7/plat-darwin', '/Users/hudanassar/anaconda/lib/python2.7/plat-mac', '/Users/hudanassar/anaconda/lib/python2.7/plat-mac/lib-scriptpackages', '/Users/hudanassar/anaconda/lib/python2.7/lib-tk', '/Users/hudanassar/anaconda/lib/python2.7/lib-old', '/Users/hudanassar/anaconda/lib/python2.7/lib-dynload', '/Users/hudanassar/anaconda/lib/python2.7/site-packages', '/Users/hudanassar/anaconda/lib/python2.7/site-packages/Sphinx-1.4.1-py2.7.egg', '/Users/hudanassar/anaconda/lib/python2.7/site-packages/aeosa', '/Users/hudanassar/anaconda/lib/python2.7/site-packages/setuptools-23.0.0-py2.7.egg']

In [4]:
@pyimport fibo

In [6]:
const z = 0
@time for i = 1:10
    z = fibo.fibIter(20)
end
@show z

  0.000168 seconds (140 allocations: 3.906 KB)


z = 6765


6765